In [44]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [45]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Event_Medikament CASCADE;
DROP TABLE IF EXISTS Arzt_Event CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
"""



In [46]:
conn.rollback()

cur.execute(drop_tables)
conn.commit()



In [47]:
create_tables = """
CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    Postleitzahl VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    Name VARCHAR(100) NOT NULL,
    CHECK (Postleitzahl ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Raum (
    Raumnummer INT PRIMARY KEY,
    Station_ID INT NOT NULL,
    Art VARCHAR(50),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflege_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    Postleitzahl VARCHAR(10),
    Stadt VARCHAR(50),
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Station_ID INT,
    CHECK (Postleitzahl IS NULL OR Postleitzahl ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Patient_ID INT,
    Raum_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    FOREIGN KEY (Raum_ID) REFERENCES Raum(Raumnummer)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Arzt_ID INT,
    Station_ID INT,
    Pflegekraft_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID),
    FOREIGN KEY (Pflegekraft_ID) REFERENCES Pflegekraft(Pflege_ID)
);

CREATE TABLE Behandlung (
    Behandlungs_ID INT PRIMARY KEY,
    abgeschlossen BOOLEAN NOT NULL,
    Beschreibung VARCHAR(255),
    Start_Datum DATE NOT NULL,
    End_Datum DATE,
    Patient_ID INT NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    CHECK (End_Datum IS NULL OR End_Datum >= Start_Datum)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Wirkstoff VARCHAR(100),
    Name VARCHAR(100) NOT NULL   
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Beschreibung VARCHAR(255),
    Uhrzeit TIME NOT NULL,
    Datum DATE NOT NULL,
    RaumID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (RaumID) REFERENCES Raum(Raumnummer),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlungs_ID)
);


CREATE TABLE Arzt_Event (
    Arzt_ID INT NOT NULL,
    Event_ID INT NOT NULL,
    PRIMARY KEY (Arzt_ID, Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID)
);

CREATE TABLE Event_Medikament (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""
cur.execute(create_tables)
conn.commit()


In [48]:
import random
import datetime
from faker import Faker

fake = Faker("de_CH")
random.seed(1)

TODAY = datetime.date(2025, 12, 12)

def phone(i):
    return f"+41 79 {i:06d}"

# -------------------------
# Konstanten
# -------------------------
BEHANDLUNG_CASES = [
    "Appendektomie Nachkontrolle",
    "Blutdruckeinstellung",
    "Diabetes Verlaufskontrolle",
    "Atemwegsinfekt Behandlung",
    "Herz-Kreislauf Check",
    "Wundversorgung",
    "Schmerztherapie"
]

EVENTS = ["Visite", "Nachkontrolle", "Medikamentengabe", "Therapieanpassung"]

STATION_NAMES = [
    "Chirurgie",
    "Innere Medizin",
    "Pädiatrie",
    "Notfall",
    "Intensivstation"
]

RAUM_LAYOUT = [
    ("Operationssaal", 1),
    ("Untersuchungsraum", 2),
    ("Patientenzimmer", 6),
    ("Aufenthaltsraum", 1)
]

MEDIKAMENTE = [
    (1, "Paracetamol", "Paracetamol"),
    (2, "Ibuprofen", "Ibuprofen"),
    (3, "Aspirin", "Acetylsalicylsäure"),
    (4, "Amoxicillin", "Amoxicillin"),
    (5, "Insulin", "Insulin")
]

# -------------------------
# 1) Krankenhäuser
# -------------------------
for kh_id in range(1, 4):
    cur.execute("""
        INSERT INTO Krankenhaus VALUES (%s,%s,%s,%s,%s,%s)
        ON CONFLICT DO NOTHING
    """, (
        kh_id,
        fake.street_name(),
        fake.building_number(),
        f"{1000 + kh_id:04d}",
        fake.city(),
        f"Kantonsspital {kh_id}"
    ))
conn.commit()

# -------------------------
# 2) Stationen
# -------------------------
station_ids = []
sid = 1
for kh_id in range(1, 4):
    for name in STATION_NAMES:
        cur.execute("""
            INSERT INTO Station VALUES (%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (sid, name, random.randint(30, 60), kh_id))
        station_ids.append(sid)
        sid += 1
conn.commit()

# -------------------------
# 3) Räume
# -------------------------
rooms_by_station = {}
sleep_rooms_by_station = {}

raum_id = 1
for s in station_ids:
    rooms_by_station[s] = []
    sleep_rooms_by_station[s] = []

    for art, count in RAUM_LAYOUT:
        for _ in range(count):
            cur.execute("""
                INSERT INTO Raum VALUES (%s,%s,%s)
                ON CONFLICT DO NOTHING
            """, (raum_id, s, art))

            rooms_by_station[s].append(raum_id)
            if art in ("Patientenzimmer", "Aufenthaltsraum"):
                sleep_rooms_by_station[s].append(raum_id)

            raum_id += 1
conn.commit()

# -------------------------
# 4) Medikamente
# -------------------------
for mid, name, wirkstoff in MEDIKAMENTE:
    cur.execute("""
        INSERT INTO Medikament VALUES (%s,%s,%s)
        ON CONFLICT DO NOTHING
    """, (mid, wirkstoff, name))
conn.commit()

# -------------------------
# 5) Patienten
# -------------------------
NUM_PATIENTS = 150
patients_by_station = {s: [] for s in station_ids}

for pid in range(1, NUM_PATIENTS + 1):
    s = random.choice(station_ids)
    patients_by_station[s].append(pid)

    cur.execute("""
        INSERT INTO Patient VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT DO NOTHING
    """, (
        pid,
        fake.first_name(),
        fake.last_name(),
        f"CH{pid:08d}",
        fake.date_of_birth(minimum_age=0, maximum_age=95),
        fake.street_name(),
        fake.building_number(),
        f"{1000 + pid % 50:04d}",
        fake.city(),
        f"EPD-{pid}",
        s
    ))
conn.commit()

# -------------------------
# 6) Ärzte (mind. 1 pro Station)
# -------------------------
doctor_ids_by_station = {}
aid = 1

for s in station_ids:
    doctor_ids_by_station[s] = []

    for _ in range(2):
        cur.execute("""
            INSERT INTO Arzt VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            aid,
            phone(aid),
            fake.first_name(),
            fake.last_name(),
            False,
            s
        ))
        doctor_ids_by_station[s].append(aid)
        aid += 1
conn.commit()

# -------------------------
# 7) Betten (nur echte Patienten)
# -------------------------
bett_id = 1

for s, rooms in sleep_rooms_by_station.items():
    pids = patients_by_station[s].copy()
    random.shuffle(pids)

    for _ in range(20):
        if pids and random.random() < 0.7:
            pid = pids.pop()
            status = "belegt"
        else:
            pid = None
            status = "frei"

        cur.execute("""
            INSERT INTO Bett VALUES (%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            bett_id,
            status,
            pid,
            random.choice(rooms)
        ))
        bett_id += 1
conn.commit()

# -------------------------
# 8) Behandlungen
# -------------------------
behandlung_id = 1
treatments_by_station = {s: [] for s in station_ids}

for s, pids in patients_by_station.items():
    for pid in pids:
        cur.execute("""
            INSERT INTO Behandlung VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            behandlung_id,
            True,
            random.choice(BEHANDLUNG_CASES),
            TODAY - datetime.timedelta(days=random.randint(5, 30)),
            TODAY,
            pid
        ))
        treatments_by_station[s].append(behandlung_id)
        behandlung_id += 1
conn.commit()

# -------------------------
# 9) Events (max 1 Event pro Behandlung pro Tag)
# -------------------------
event_id = 1

for s, bids in treatments_by_station.items():
    for bid in bids:
        ev = random.choice(EVENTS)
        raum = random.choice(rooms_by_station[s])

        cur.execute("""
            INSERT INTO Event VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT DO NOTHING
        """, (
            event_id,
            ev,
            fake.time(),
            TODAY,
            raum,
            bid
        ))

        arzt = random.choice(doctor_ids_by_station[s])
        cur.execute("""
            INSERT INTO Arzt_Event VALUES (%s,%s)
            ON CONFLICT DO NOTHING
        """, (arzt, event_id))

        event_id += 1
conn.commit()

# -------------------------
# 10) Event_Medikament
# -------------------------
cur.execute("SELECT event_id FROM Event")
for (eid,) in cur.fetchall():
    for mid in random.sample([1,2,3,4,5], random.randint(1,2)):
        cur.execute("""
            INSERT INTO Event_Medikament VALUES (%s,%s)
            ON CONFLICT DO NOTHING
        """, (eid, mid))
conn.commit()

print("✔ Datenbank sauber, logisch & konsistent befüllt")


✔ Datenbank sauber, logisch & konsistent befüllt


In [49]:
conn.rollback()
